In [192]:
from requests import get, post
from bs4 import BeautifulSoup
from pandas import DataFrame

In [3]:
def getTable(html):
    itemData = {"Name": [], "License Number": [], "Original Issue Date": [], "Expiration": []}
    item_soup = BeautifulSoup(html, 'html.parser')

    containers = item_soup.find_all("tr")
    for i in containers[2:]:
        l = i.find_all("td")
        if len(l) != 8:continue
        l.pop(2)
        l.pop(2)
        name, number, issue, expiration, *other = [j.text for j in l]
        itemData["Name"].append(name)
        itemData["License Number"].append(number)
        itemData["Original Issue Date"].append(issue)
        itemData["Expiration"].append(expiration)
    
    return DataFrame(itemData)


In [189]:
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from pandas import read_html

def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response


def scrape():
    caps = DesiredCapabilities.CHROME
    caps['goog:loggingPrefs'] = {'performance': 'ALL'}
    driver = webdriver.Chrome(desired_capabilities=caps)

    driver.get("https://nrec.igovsolution.net/online/Verification/Individual")

    links = []

    # driver.implicitly_wait(10)

    captcha = input()
    captcha_box = driver.find_element(by=By.ID, value="ctl00_cntbdy_txt_verify")
    driver.find_element(by=By.ID, value="ctl00_cntbdy_ddllictype").send_keys("Broker")
    driver.find_element(by=By.ID, value="ctl00_cntbdy_ddllicstatus").send_keys("Active")
    captcha_box.send_keys(captcha)

    submit_button = driver.find_element(by=By.ID, value="ctl00_cntbdy_btn_search")
    submit_button.click()
    for i in range(40):
        print(i)
        links = []
        # input("enter to continue")
        WebDriverWait(driver, 80).until(EC.presence_of_element_located((By.ID, "grdlkp_tbl")))
        # driver.find_elements(By.TAG_NAME, "select")[-2].send_keys("100")
        table = driver.find_element(by=By.ID, value="grdlkp_tbl")
        # display(getTable(table.get_attribute('innerHTML')))
        # WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "fa-print")))
        buttons = driver.find_elements(by=By.CLASS_NAME, value="fa-print")
        close = driver.find_element(by=By.CLASS_NAME, value="fa-close")

        for button in buttons:
            for test in range(3):
                try:
                    button.click()
                    break
                except:
                    print("stale error")
                    continue
            # WebDriverWait(driver, 10)
            # driver.find_elements(By.TAG_NAME, "select")[-2].send_keys(10)
            browser_log = driver.get_log('performance') 
            events = [process_browser_log_entry(entry) for entry in browser_log]
            events = [event for event in events if 'Network.response' in event['method']]
            for req in browser_log:
                if "https://nrec.igovsolution.net/online/printverification" in str(req):
                    try:
                        # print(json.loads(req['message'])["message"]["params"]["url"])
                        links.append(json.loads(req['message'])["message"]["params"]["url"])
                        # print()
                    except:pass
            # iframe = driver.find_element(by=By.ID, value="frm")
            # print(">>>"+iframe.text)
            # WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "frm")))
            # driver.switch_to.frame("frm")
            # print(driver.page_source)
            # WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.ID, "form1")))
            # details = driver.find_element(by=By.ID, value="form1")
            # print(details.get_attribute("innerHTML"))
            # detailsDF = read_html(driver.page_source)[0]
            # display(detailsDF)
            # driver.switch_to.default_content()
            for test in range(3):
                try:
                    WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME, "fa-close")))
                    close.click()
                    break
                except:
                    print("non interactable error")
                    continue
            # WebDriverWait(driver, 2)
        # for test in range(3):
        #     try:
        #         WebDriverWait(driver,20).until(EC.element_to_be_clickable((By.CLASS_NAME, "fa-angle-right")))
        #         driver.find_element(By.CLASS_NAME, "fa-angle-right").click()
        #         print(">"*50)
        #         break
        #     except:
        #         print("click intercepted error")
        #         continue
        print("*"*50)
        print(set(links))
        with open('network.txt', 'a') as f:
            f.write("\n"+str(set(links)))
        WebDriverWait(driver,60).until(EC.element_to_be_clickable((By.ID, "ctl00_cntbdy_btn_search")))
        submit_button.click()
    
    driver.quit()
    
scrape()

0
stale error
stale error
stale error
**************************************************
{'https://nrec.igovsolution.net/online/printverification/22014-24215', 'https://nrec.igovsolution.net/online/printverification/22742-25059', 'https://nrec.igovsolution.net/online/printverification/21948-24143', 'https://nrec.igovsolution.net/online/printverification/22082-24299', 'https://nrec.igovsolution.net/online/printverification/21926-24117', 'https://nrec.igovsolution.net/online/printverification/22296-24549', 'https://nrec.igovsolution.net/online/printverification/23505-25910', 'https://nrec.igovsolution.net/online/printverification/21925-24116', 'https://nrec.igovsolution.net/online/printverification/22073-24288', 'https://nrec.igovsolution.net/online/printverification/22409-24681', 'https://nrec.igovsolution.net/online/printverification/22269-24517', 'https://nrec.igovsolution.net/online/printverification/22250-24495', 'https://nrec.igovsolution.net/online/printverification/22655-24961', 

ElementClickInterceptedException: Message: element click intercepted: Element <input type="submit" name="ctl00$cntbdy$btn_search" value="Search" onclick="javascript:return Validate_Search();" id="ctl00_cntbdy_btn_search" class="action-button" style="padding: 5px"> is not clickable at point (483, 17). Other element would receive the click: <div class="ui-widget-overlay ui-front" style="z-index: 9999;"></div>
  (Session info: chrome=108.0.5359.124)
Stacktrace:
0   chromedriver                        0x000000010096cf38 chromedriver + 4910904
1   chromedriver                        0x00000001008eca03 chromedriver + 4385283
2   chromedriver                        0x0000000100531747 chromedriver + 472903
3   chromedriver                        0x000000010057f588 chromedriver + 791944
4   chromedriver                        0x000000010057c9ec chromedriver + 780780
5   chromedriver                        0x0000000100579671 chromedriver + 767601
6   chromedriver                        0x0000000

In [190]:
with open('network.txt') as f:
    lines = f.readlines()
    l = []
    for i in lines:
        l+=list(i.replace("{","").replace("}","").replace(" ","").split(","))
    print(len(set(l)))

2004


In [191]:
arr = list(set(l))
itemData = {"Name": [], "License Number": [], "Original Issue Date": [], "Expiration": [], "Phone Number": [], "Email": [], "Address": []}
for i in arr:
    print(i.strip().replace("'","").replace("{","").replace("}",""))
    html = get(i.strip().replace("'","").replace("{","").replace("}",""))
    soup = BeautifulSoup(html.text, 'html.parser')
    containers = soup.find_all("td")
    for j in containers:
        a = j.text
        if "Name :" in a:itemData["Name"].append(a.replace("Name :", "").strip())
        elif "License No :" in a:itemData["License Number"].append(a.replace("License No :", "").strip())
        elif "Date Issued :" in a:itemData["Original Issue Date"].append(a.replace("Date Issued :", "").strip())
        elif "Expiration Date :" in a:itemData["Expiration"].append(a.replace("Expiration Date :", "").strip())
        elif "Phone :" in a:itemData["Phone Number"].append(a.replace("Phone :", "").strip())
        elif "Email :" in a:itemData["Email"].append(a.replace("Email :", "").strip())
        elif "Address :" in a:itemData["Address"].append(a.replace("Address :", "").strip())
df = DataFrame(itemData)
display(df)
df.to_csv("nebraskaAgentsUpdated.csv")


https://nrec.igovsolution.net/online/printverification/26828-29693
https://nrec.igovsolution.net/online/printverification/9561-9879
https://nrec.igovsolution.net/online/printverification/20281-22240
https://nrec.igovsolution.net/online/printverification/3851-3876
https://nrec.igovsolution.net/online/printverification/3534-3555
https://nrec.igovsolution.net/online/printverification/11157-11705
https://nrec.igovsolution.net/online/printverification/21568-23688
https://nrec.igovsolution.net/online/printverification/20411-22387
https://nrec.igovsolution.net/online/printverification/3813-3838
https://nrec.igovsolution.net/online/printverification/10730-11213
https://nrec.igovsolution.net/online/printverification/8497-8710
https://nrec.igovsolution.net/online/printverification/7467-7609
https://nrec.igovsolution.net/online/printverification/2308-2309
https://nrec.igovsolution.net/online/printverification/26379-131493
https://nrec.igovsolution.net/online/printverification/29497-32451
https://

,Name,License Number,Original Issue Date,Expiration,Phone Number,Email,Address
0,Stephen Christopher Leick,20190757,10/28/2019,12/31/2023,(402) 990-4750,[email protected],5103 HAMILTON ST OMAH...
1,Mark Jeffrey Wiesman,0940008,01/01/1994,12/31/2024,(402) 398-1111,[email protected],1000 N 90TH ST STE 100 ...
2,Carole Louise Souza,20090594,11/16/2009,12/31/2023,(402) 932-5989,[email protected],1411 N 128TH CIR OMAH...
3,Steve Kettelhut,0800300,03/17/1980,12/31/2023,(402) 729-3060,[email protected],515 E St PO Box 516 ...
4,Kenneth A Cudly,0790059,03/26/1979,12/31/2023,4029101559,[email protected],9800 Hollow Tree Dr L...
...,...,...,...,...,...,...,...
1998,Ruth Lynette Beahm,0980484,08/13/1998,12/31/2024,3088322444,[email protected],PO Box 275 2350 N Br...
1999,Donald W Linscott,0730242,10/30/1973,12/31/2023,4024672525,[email protected],"300 N 44, Ste 112 Lin..."
2000,Jill Nicole Smith,20200069,01/17/2020,12/31/2024,(402) 564-9636,[email protected],2872 54TH AVE COLUMBU...
2001,Stacey Rae Jaeger,20160173,01/28/2016,12/31/2024,(402) 802-4377,[email protected],10351 N 143RD ST WAVE...


In [197]:
post("http://upserter.products.stackendsolutions.com/message", data={
                                                            "destination_type": "sql",
                                                            "connect_url": "stackend",
                                                            "table": "NebraskaRealEstateAgents",
                                                            "object": itemData,
                                                            "search": {"ID": 1},
                                                            })

<Response [404]>